In [1]:
import sys
sys.path.append("../../common")

In [37]:
import aoc
from collections import deque
import heapq
import numpy as np
import re

In [122]:
flow_rates = {}
links = {}
with open("./puzzle_inputs/16.txt") as f:
    for line in f:
        tunnel, flow_rate, *linked_tunnels = re.findall("([A-Z]{2}|\d+)", line)
        flow_rates[tunnel] = int(flow_rate)
        links[tunnel] = linked_tunnels
valve_ids = {link: index for index, link in enumerate(links)}
valve_map = {v: k for k, v in valve_ids.items()}

In [123]:
flow_rates = np.array([flow_rates[valve] for valve in valve_ids])
flow_rates

array([ 0,  0,  0,  0, 10,  0, 15,  0,  0,  0,  0,  0,  0, 20, 19, 14,  0,
        0, 18,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0, 17, 24,
        0,  0,  0,  0, 22,  8,  4,  0,  0, 21,  0,  0,  0,  0,  0,  0,  0,
        7,  0,  0,  0,  0,  0,  0, 11,  0])

In [165]:
distance_mat = np.zeros((len(links), len(links)), dtype=int)
for start in links:
    visited = set()
    queue = deque([(start, 0)])
    while len(queue) > 0:
        tunnel, distance = queue.popleft()
        visited.add(tunnel)
        i, j = valve_ids[start], valve_ids[tunnel]
        distance_mat[i,j] = distance + 1
        for link in links[tunnel]:
            if link in visited:
                continue
            queue.append((link, distance + 1))
    len(visited)
# distance_mat = distance_mat[valves_worth_opening][:,valves_worth_opening]
distance_mat.shape

(60, 60)

In [205]:
valves_worth_opening = set([valve for valve, flow_rate in enumerate(flow_rates) if flow_rate > 0])
valves_worth_opening

{4, 6, 13, 14, 15, 18, 27, 32, 33, 38, 39, 40, 43, 51, 58}

In [206]:
distance_mat

array([[ 1,  3,  8, ...,  6, 14, 12],
       [ 3,  1,  9, ...,  8, 16, 14],
       [ 8,  8,  1, ...,  5, 11,  9],
       ...,
       [ 6,  8,  6, ...,  1, 11,  9],
       [14, 16, 11, ..., 11,  1,  3],
       [12, 14,  9, ...,  9,  3,  1]])

In [207]:
time_mat = np.full_like(distance_mat, 30)
cost_mat = np.zeros_like(distance_mat)
for i in range(len(cost_mat)):
    for j in range(len(cost_mat)):
        # Update the current row
        time_remaining = time_mat[i][j] - distance_mat[i][j]
        cost = time_remaining*flow_rates[j] + cost_mat[i][j]
        if cost <= cost_mat[i][j]:
            continue
        cost_mat[i][j] = cost
        # Update other rows
        for k in range(len(cost_mat)):
            if cost >= cost_mat[j][k]:
                cost_mat[j][k] = cost
                time_mat[j][k] = time_remaining

In [199]:
cost_mat[0]

array([  0,   0,   0,   0, 280,   0, 375,   0,   0,   0,   0,   0,   0,
       500, 475, 364,   0,   0, 252,   0,   0,   0,   0,   0,   0,   0,
         0,  69,   0,   0,   0,   0, 374, 624,   0,   0,   0,   0, 616,
       184,  76,   0,   0, 546,   0,   0,   0,   0,   0,   0,   0, 161,
         0,   0,   0,   0,   0,   0, 176,   0])

In [159]:
for i in range(len(cost_mat)):
    if i == row:
        continue
    current = cost_mat[0][i]
    if current == 0:
        continue
    time_remaining = cost_mat[0][i] // flow_rates[i]
    for j in range(len(cost_mat)):
        new_cost = (time_remaining - distance_mat[i][j])*flow_rates[j]
        if new_cost < 0:
            continue
        cost_mat[i][j] = max(cost_mat[i][j], new_cost + current)

In [160]:
np.max(cost_mat)

624

In [139]:
for i in np.where(cost_mat[0] > 0)[0]:
    cost_mat[i] = cost_mat[0][i]

In [140]:
cost_mat[4]

array([280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280,
       280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280,
       280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280,
       280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280,
       280, 280, 280, 280, 280, 280, 280, 280])

In [208]:
from time import time as current_seconds

In [212]:
heap = [(0, 0, valve_ids["AA"], set())] # (total_pressure, total_time, valve)
s = current_seconds()
max_pressure = 0
best_path = None
while len(heap) > 0:
    if current_seconds() - s > 1:
        print("\r", len(heap), end="")
        s = current_seconds()
    total_pressure_neg, total_time, current_valve, opened_valves = heapq.heappop(heap)
    if -total_pressure_neg > max_pressure:
        best_path = opened_valves
    max_pressure = max(max_pressure, -total_pressure_neg)
    for valve in valves_worth_opening - opened_valves:
    # for valve in valves_worth_opening:
        if valve in opened_valves:
            continue
        time = total_time + distance_mat[current_valve, valve]
        if time > 30:
            continue
        pressure = -total_pressure_neg + (30 - time)*flow_rates[valve]
        heapq.heappush(heap, (-pressure, time, valve, opened_valves | set([valve])))
        # heapq.heappush(heap, (-pressure, time, valve, opened_valves + [valve]))


 30

In [213]:
max_pressure

2124